# Documentación técnica — Etapa “Recalls → Limpieza + Export + Ingesta Neo4j (Aura)”

> Esta etapa prepara el dataset de **recalls NHTSA** desde la fuente “flat” comprimida, aplica la **misma limpieza y controles** del EDA original (Plan A/B, mapeo Appendix A), **descompone la jerarquía de componentes**, y lo **ingesta** en un **grafo único** en Neo4j Aura con relaciones y jerarquía de `Component`.

---

## 1) Entradas / Salidas

**Entradas**

* `drive/MyDrive/NHTSA/source/FLAT_RCL_POST_2010.zip`
  Contiene el TXT tab-delimited con los recalls.

**Artefactos intermedios**

* `drive/MyDrive/NHTSA/processed/recalls_raw_enriched.csv` (dump tras limpieza y tipificación)
* `drive/MyDrive/NHTSA/processed/recalls_by_campaign.csv` (agregado por campaña)

**Salida para ingesta (definitiva)**

* `drive/MyDrive/NHTSA/neo4j/exports/recalls_neo4j_ready.csv`

**Resultado en Neo4j Aura (conteos observados)**

* `(:Recall)` = **14,340** nodos
* `(:Component)` = **509** nodos
* Jerarquía `(:Component)-[:SUB_OF]->(:Component)` creada (hasta 5 niveles)
* Enlaces `(:Recall)-[:MENTIONS]->(:Component)` al nivel **más específico disponible**
* Enlaces `(:Recall)-[:OF_MAKE]->(:Make)` y `(:Recall)-[:OF_MODEL]->(:Model)`

---

## 2) Lectura robusta (Plan A/B) y mapeo Appendix A

La fuente “flat” presenta **inconsistencias de comillas** en narrativas largas. Se implementó un lector **Plan A/B**:

* **Plan A (fiel):** `quotechar='"'` y `csv.QUOTE_MINIMAL`
  Falló con el error clásico: `' \t ' expected after '"'` → comillas no escapadas.
* **Plan B (tolerante):** `quoting=csv.QUOTE_NONE`, `on_bad_lines="warn"`
  Éxito: **222,573 filas**, **29 columnas**.

Se mapean **estrictamente** las primeras **24 columnas** al **Appendix A**:

```text
[1..24] → ['RECORD_ID','CAMPNO','MAKETXT','MODELTXT','YEARTXT','MFGCAMPNO','COMPNAME','MFGNAME',
           'BGMAN','ENDMAN','RCLTYPECD','POTAFF','ODATE','INFLUENCED_BY','MFGTXT','RCDATE',
           'DATEA','RPNO','FMVSS','DESC_DEFECT','CONEQUENCE_DEFECT','CORRECTIVE_ACTION',
           'NOTES','RCL_CMPT_ID']
```

* Columnas extra ≥25 se preservan con prefijo `EXTRA_XX` para **trazabilidad**.
* Se corrige el **typo** `CONEQUENCE_DEFECT` → `CONSEQUENCE_DEFECT`.

**Rationale**

* Mantener la semántica original de NHTSA y reproducibilidad.
* Evitar “adivinar” columnas; el mapeo es **determinista** y auditable.

---

## 3) Normalización y tipificación

* `YEARTXT`: entero **[1950, 2035]**; `9999` → `NaN`.
* Fechas: `ODATE`, `RCDATE`, `DATEA`, `BGMAN`, `ENDMAN` → `datetime` con formato `%Y%m%d`.
* `POTAFF` → `POTAFF_num` (numérico auxiliar).
* Narrativas (`DESC_DEFECT`, `CONSEQUENCE_DEFECT`, `CORRECTIVE_ACTION`, `NOTES`):

  * **Limpieza superficial de HTML**, normalización de espacios.
  * Se calculan longitudes (`len_*`) como métricas de calidad.

**Rationale**

* Rango de años evita outliers y años centinela.
* Limpieza de HTML reduce ruido para posteriores embeddings y búsqueda textual.

---

## 4) Jerarquía de componentes

* Campo `COMPNAME` se **descompone por `:`** en **hasta 5 niveles**: `COMP_L1..COMP_L5`.
* Se eliminan literales vacíos y “`NONE`”.

**Rationale**

* La jerarquía soporta consultas semánticas por **nivel de granularidad** y habilita la relación `SUB_OF` entre componentes.

---

## 5) De-duplicación por campaña y agregado

* **De-dup:** Se ordena por (`CAMPNO`, `RCDATE`) y se conserva el **registro más reciente** por campaña.
* **Agregado por `CAMPNO`:**

  * Métricas agregadas:
    `POTAFF_num=max`, fechas (`RCDATE/ODATE/DATEA`) mínimas, `ENDMAN` máxima, `YEARTXT=max`
    Atributos descriptivos: `MAKETXT/MODELTXT/COMPNAME/COMP_L*` → `first`
* Resultado: `recalls_by_campaign.csv`

**Rationale**

* Evita **sobrecontar** campañas con múltiples filas.
* `max` de `POTAFF_num` y fechas mín/max preservan una vista **conservadora** y coherente.

---

## 6) Controles de calidad (observados en esta corrida)

* **Cobertura de fechas (por campaña)**
  `RCDATE`: **100.0%** | `ODATE`: **97.75%** | `DATEA`: **100%**
* **Narrativas (crudo)**
  `DESC_DEFECT/CONSEQUENCE_DEFECT/CORRECTIVE_ACTION`: **100%** | `NOTES`: **97.3%**
* **POTAFF**
  `min=0`, `max=17,600,000`, `mediana=466`, `≤0`: **6** campañas
* **YEARTXT**
  Rango **[1976, 2026]**

**Rationale**

* Estos checks replican los del EDA original, garantizando **consistencia** y **calidad mínima** antes del grafo.

---

## 7) Export “listo para Aura”

Se genera `neo4j/exports/recalls_neo4j_ready.csv` con columnas:

```text
['campaign_no','make','model','year','component','recall_date',
 'subject','consequence','comp_l1','comp_l2','comp_l3','comp_l4','comp_l5']
```

* `campaign_no` = `CAMPNO`
* `make/model` = `MAKETXT/MODELTXT` normalizados (uppercase, espacios compactados)
* `year` = `YEARTXT` (nullable)
* `component` = original (`COMPNAME`) si existe; fallback a `comp_l1`
* `recall_date` = ISO `YYYY-MM-DD` (string; vacío si no parseable)
* `comp_l*` = jerarquía por `:`

**Decisión importante:** **no se exige `year`** en el filtro mínimo del export para **no vaciar** el dataset en casos de año faltante. Se garantizan `campaign_no`, `make`, `model` y `comp_l1`.

**Resultado:** **14,340** filas exportadas.

---

## 8) Ingesta en Neo4j Aura (constraints, índices, upsert)

### 8.1 Constraints / Índices

Se crean **una por una** (Aura requiere 1 statement por query):

```cypher
CREATE CONSTRAINT IF NOT EXISTS FOR (r:Recall)        REQUIRE r.id IS UNIQUE;
CREATE CONSTRAINT IF NOT EXISTS FOR (i:Investigation) REQUIRE i.id IS UNIQUE;
CREATE CONSTRAINT IF NOT EXISTS FOR (c:Complaint)     REQUIRE c.id IS UNIQUE;
CREATE CONSTRAINT IF NOT EXISTS FOR (c:Component)     REQUIRE c.name IS UNIQUE;
CREATE CONSTRAINT IF NOT EXISTS FOR (m:Make)          REQUIRE m.name IS UNIQUE;
CREATE CONSTRAINT IF NOT EXISTS FOR (m:Model)         REQUIRE (m.name, m.make) IS UNIQUE;
CREATE CONSTRAINT IF NOT EXISTS FOR (x:Issue)         REQUIRE x.id IS UNIQUE;

CREATE INDEX comp_name_lower IF NOT EXISTS FOR (c:Component) ON (c.name_lower);
```

**Rationale**

* **Unicidad** por dominio evita duplicados en upserts idempotentes.
* Índice `name_lower` acelera comparaciones case-insensitive si se requieren.

### 8.2 Cypher de upsert (compatible con Aura)

Aura no permite `CALL { WITH ... WHERE ... }` para “importing WITH”. Se usa patrón `FOREACH` condicional (listas vacías) y un `CASE` para seleccionar el nivel objetivo de `MENTIONS`.

```cypher
UNWIND $rows AS row
MERGE (r:Recall {id: row.campaign_no})
  SET r.camp_no     = row.campaign_no,
      r.recall_date = CASE WHEN row.recall_date = '' THEN NULL ELSE row.recall_date END,
      r.make        = row.make,
      r.model       = row.model,
      r.year        = CASE WHEN row.year IS NULL OR row.year = '' THEN NULL ELSE toInteger(row.year) END,
      r.component   = row.component,
      r.subject     = coalesce(row.subject, ''),
      r.consequence = coalesce(row.consequence, '')

MERGE (mk:Make {name: row.make})
MERGE (md:Model {name: row.model, make: row.make})
MERGE (r)-[:OF_MAKE]->(mk)
MERGE (r)-[:OF_MODEL]->(md)

// Jerarquía Component
MERGE (c1:Component {name: row.comp_l1})
  ON CREATE SET c1.name_lower = toLower(row.comp_l1)
  ON MATCH  SET c1.name_lower = coalesce(c1.name_lower, toLower(row.comp_l1))

FOREACH (_ IN CASE WHEN row.comp_l2 <> '' THEN [1] ELSE [] END |
  MERGE (p1:Component {name: row.comp_l1})
  MERGE (c2:Component {name: row.comp_l2})
    ON CREATE SET c2.name_lower = toLower(row.comp_l2)
    ON MATCH  SET c2.name_lower = coalesce(c2.name_lower, toLower(row.comp_l2))
  MERGE (c2)-[:SUB_OF]->(p1)
)

FOREACH (_ IN CASE WHEN row.comp_l3 <> '' AND row.comp_l2 <> '' THEN [1] ELSE [] END |
  MERGE (p2:Component {name: row.comp_l2})
  MERGE (c3:Component {name: row.comp_l3})
    ON CREATE SET c3.name_lower = toLower(row.comp_l3)
    ON MATCH  SET c3.name_lower = coalesce(c3.name_lower, toLower(row.comp_l3))
  MERGE (c3)-[:SUB_OF]->(p2)
)

FOREACH (_ IN CASE WHEN row.comp_l4 <> '' AND row.comp_l3 <> '' THEN [1] ELSE [] END |
  MERGE (p3:Component {name: row.comp_l3})
  MERGE (c4:Component {name: row.comp_l4})
    ON CREATE SET c4.name_lower = toLower(row.comp_l4)
    ON MATCH  SET c4.name_lower = coalesce(c4.name_lower, toLower(row.comp_l4))
  MERGE (c4)-[:SUB_OF]->(p3)
)

FOREACH (_ IN CASE WHEN row.comp_l5 <> '' AND row.comp_l4 <> '' THEN [1] ELSE [] END |
  MERGE (p4:Component {name: row.comp_l4})
  MERGE (c5:Component {name: row.comp_l5})
    ON CREATE SET c5.name_lower = toLower(row.comp_l5)
    ON MATCH  SET c5.name_lower = coalesce(c5.name_lower, toLower(row.comp_l5))
  MERGE (c5)-[:SUB_OF]->(p4)
)

// MENTIONS: nivel más profundo disponible
WITH r,
     CASE
       WHEN row.comp_l5 <> '' THEN row.comp_l5
       WHEN row.comp_l4 <> '' THEN row.comp_l4
       WHEN row.comp_l3 <> '' THEN row.comp_l3
       WHEN row.comp_l2 <> '' THEN row.comp_l2
       ELSE row.comp_l1
     END AS target_comp
MATCH (cx:Component {name: target_comp})
MERGE (r)-[:MENTIONS]->(cx)

RETURN count(r) AS upserted;
```

**Rationale**

* `FOREACH` evita restricciones de Aura sobre `WITH` importado; patrón **100% compatible**.
* `CASE` centraliza la elección del **nivel objetivo** para `MENTIONS`.
* `MERGE` asegura **idempotencia** (re-ejecuciones seguras).

### 8.3 Ingesta por lotes (rate-limit friendly)

* Tamaño de lote usado: **400** (si aparece rate limit, bajar a **200**).
* Proceso:

  1. Test con 200 filas → verificación de conteos.
  2. Ingesta completa.
  3. Validaciones post-ingesta (muestras, conteos).

**Conteos finales observados**

* `Recalls:` **14,340**
* `Components:` **509**

---

## 9) Validaciones en Neo4j (útiles)

Componentes enlazados a Recalls:

```cypher
MATCH (r:Recall)-[:MENTIONS]->(c:Component)
RETURN r.id AS campaign, c.name AS component
LIMIT 10;
```

Jerarquía de componentes:

```cypher
MATCH (c2:Component)-[:SUB_OF]->(c1:Component)
RETURN c1.name AS L1, collect(DISTINCT c2.name)[0..10] AS children
LIMIT 10;
```

Recalls por componente:

```cypher
MATCH (r:Recall)-[:MENTIONS]->(c:Component {name:'POWER TRAIN'})
RETURN count(r) AS recalls_powertrain;
```

---

## 10) Decisiones de diseño (por qué son las mejores aquí)

1. **Plan A/B de lectura**

   * **A** conserva fidelidad cuando hay comillas válidas; **B** garantiza **robustez** ante datos sucios sin perder filas (`on_bad_lines="warn"`).
   * Minimiza fallas del pipeline y asegura **cobertura total**.

2. **Mapeo fijo (Appendix A) y preservación de extras**

   * Evita heurísticas “adivinadas”; asegura **reproducibilidad** y **auditoría**.
   * Las `EXTRA_XX` mantienen trazabilidad para etapas avanzadas.

3. **Normalización conservadora**

   * Rango de `YEARTXT`, limpieza HTML y parseo de fechas estandariza sin alterar semántica.

4. **Jerarquía de `Component` por “:”**

   * Representa la **estructura real** de NHTSA (componente/subcomponente).
   * Habilita consultas multi-nivel y razonamiento causal.

5. **De-dup + agregado por `CAMPNO`**

   * Evita **duplicidades** por campaña y sobreconteos, manteniendo coherencia temporal.

6. **Export con `year` no obligatorio**

   * Evita **vaciar** el dataset por años faltantes; se preserva información crítica de campaña y componente.

7. **Cypher sin subconsultas con `WITH WHERE`**

   * Patrón `FOREACH` es **compatible con Aura**, robusto e idempotente.
   * Mantiene la lógica de jerarquía y el enlace al nivel más profundo sin hacks.

8. **Constraints/Índices**

   * Integridad (unicidad) + performance razonable en **Aura Free**.

9. **Ingesta por lotes**

   * Respeta **rate limits** de Aura Free; balancea throughput y confiabilidad.

---

## 11) Reproducibilidad (checklist)

* [x] `source/FLAT_RCL_POST_2010.zip` presente.
* [x] Lectura Plan A/B ejecutada y log impreso.
* [x] `processed/recalls_raw_enriched.csv` y `processed/recalls_by_campaign.csv` generados.
* [x] Export final `neo4j/exports/recalls_neo4j_ready.csv` con `comp_l1..comp_l5`.
* [x] Constraints/índices creados **uno por query**.
* [x] Ingesta por lotes (test 200 → full).
* [x] Conteos verificados (`Recalls`, `Components`).
* [x] Consultas de validación de jerarquía y `MENTIONS`.

---

## 12) Apéndice — Snippets mínimos

**Lectura Plan A/B desde ZIP (resumen):**

```python
with zipfile.ZipFile(SRC_ZIP, 'r') as zf:
    txt = [n for n in zf.namelist() if n.lower().endswith('.txt')][0]
    raw = io.BytesIO(zf.read(txt))

try:
    df = pd.read_csv(raw, sep="\t", header=None, engine="python",
                     dtype=str, quotechar='"', quoting=csv.QUOTE_MINIMAL,
                     on_bad_lines="error")
except Exception:
    raw.seek(0)
    df = pd.read_csv(raw, sep="\t", header=None, engine="python",
                     dtype=str, quoting=csv.QUOTE_NONE,
                     on_bad_lines="warn")
```

**Jerarquía de componentes (split por `:`):**

```python
parts = df['COMPNAME'].fillna('').astype(str).str.split(':', n=4, expand=True)
parts.columns = ['COMP_L1','COMP_L2','COMP_L3','COMP_L4','COMP_L5']
```

**Cypher Aura-friendly (FOREACH + CASE) → Recalls:**

> (idéntico al bloque 8.2)

---

Con esta etapa cerrada, el grafo tiene el **núcleo de Recalls** sólido, **jerárquico** y **consultable**. A partir de aquí podemos añadir:

* `Investigation` → `RESULTED_IN` → `Recall` (enlace por `CAMPNO`).
* `Complaint` → `Issue` (clusters) y `MENTIONS` → `Component`.
* Capa semántica externa (Qdrant) y orquestación (FastAPI) para el agente.


In [ ]:
    !apt-get install tree


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (123 kB/s)
Selecting previously unselected package tree.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!tree drive/MyDrive/NHTSA/

drive/MyDrive/NHTSA/
├── embeddings
│   ├── complaints_e5_mlg_instruct
│   │   ├── checkpoint.json
│   │   ├── embeddings_shard_0000.npy
│   │   ├── embeddings_shard_0001.npy
│   │   ├── embeddings_shard_0002.npy
│   │   ├── embeddings_shard_0003.npy
│   │   ├── embeddings_shard_0004.npy
│   │   ├── embeddings_shard_0005.npy
│   │   ├── embeddings_shard_0006.npy
│   │   ├── embeddings_shard_0007.npy
│   │   ├── embeddings_shard_0008.npy
│   │   ├── embeddings_shard_0009.npy
│   │   ├── embeddings_shard_0010.npy
│   │   ├── manifest.json
│   │   ├── meta_shard_0000.parquet
│   │   ├── meta_shard_0001.parquet
│   │   ├── meta_shard_0002.parquet
│   │   ├── meta_shard_0003.parquet
│   │   ├── meta_shard_0004.parquet
│   │   ├── meta_shard_0005.parquet
│   │   ├── meta_shard_0006.parquet
│   │   ├── meta_shard_0007.parquet
│   │   ├── meta_shard_0008.parquet
│   │   ├── meta_shard_0009.parquet
│   │   └── meta_shard_0010.parquet
│   ├── investigations_e5_mlg_instruct
│   │   ├── config.jso

In [ ]:
!pip -q install neo4j pandas pyarrow python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.8/325.8 kB 6.3 MB/s eta 0:00:00


In [ ]:
from pathlib import Path

BASE = Path('/content/drive/MyDrive/NHTSA')
EMB  = BASE / 'embeddings'
PROC = BASE / 'processed'
EXPORTS = BASE / 'neo4j' / 'exports'
EXPORTS.mkdir(parents=True, exist_ok=True)
list(BASE.glob('**/*'))[:5], BASE.exists()

([PosixPath('/content/drive/MyDrive/NHTSA/processed'),
  PosixPath('/content/drive/MyDrive/NHTSA/embeddings'),
  PosixPath('/content/drive/MyDrive/NHTSA/neo4j'),
  PosixPath('/content/drive/MyDrive/NHTSA/processed/recalls_corpus.jsonl'),
  PosixPath('/content/drive/MyDrive/NHTSA/processed/complaints.parquet')],
 True)

## Credenciales de Neo4j AuraDB


In [ ]:
import os
NEO4J_URI  = os.getenv('NEO4J_URI',  'neo4j+s://66024f48.databases.neo4j.io')
NEO4J_USER = os.getenv('NEO4J_USER', 'neo4j')
NEO4J_PASS = os.getenv('NEO4J_PASS', 'kDp50qsUISmBomZa8F9htkq-s5zcb-rlxbgyKYzdVEI')
print(NEO4J_URI, NEO4J_USER)

neo4j+s://66024f48.databases.neo4j.io neo4j


## Construir `recalls_neo4j_ready.csv` desde tus archivos en `processed/`


In [ ]:
# ==========================================
# NHTSA Recalls → limpieza “EDA original” + export Neo4j
# Origen: drive/MyDrive/NHTSA/source/FLAT_RCL_POST_2010.zip
# Salidas:
#   processed/recalls_raw_enriched.csv
#   processed/recalls_by_campaign.csv
#   neo4j/exports/recalls_neo4j_ready.csv
# ==========================================
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd, numpy as np, csv, re, zipfile, io
from pathlib import Path

BASE     = Path('/content/drive/MyDrive/NHTSA')
SRC_ZIP  = BASE / 'source' / 'FLAT_RCL_POST_2010.zip'
PROC_DIR = BASE / 'processed'
EXP_DIR  = BASE / 'neo4j' / 'exports'
PROC_DIR.mkdir(parents=True, exist_ok=True)
EXP_DIR.mkdir(parents=True, exist_ok=True)

assert SRC_ZIP.exists(), f"No encuentro: {SRC_ZIP}"

# --------- utilidades ----------
def strip_html_series(s: pd.Series) -> pd.Series:
    def _clean(t):
        if not isinstance(t, str): return t
        t = re.sub(r"<\s*/?\s*a\b[^>]*>", " ", t, flags=re.I)  # quita <a>
        t = re.sub(r"<[^>]+>", " ", t)                         # quita cualquier tag
        return re.sub(r"\s+", " ", t).strip()
    return s.fillna("").map(_clean)

def split_component_hierarchy(s: pd.Series, max_levels: int = 5) -> pd.DataFrame:
    """divide COMPNAME por ':' en hasta max_levels niveles, sin dejar 'None' literales."""
    vals = s.fillna('').astype(str).tolist()
    out = []
    for v in vals:
        parts = [p.strip() for p in v.split(':') if isinstance(p,str)]
        parts = [p for p in parts if p and p.upper() != 'NONE']
        parts = parts[:max_levels] + ['']*(max_levels-len(parts))
        out.append(parts)
    cols = [f'COMP_L{i}' for i in range(1, max_levels+1)]
    return pd.DataFrame(out, columns=cols, index=s.index)

# --------- lector Plan A/B DESDE ZIP ----------
# El ZIP puede traer 1 TXT (FLAT_RCL_POST_2010.txt). Lo detectamos y leemos su contenido.
with zipfile.ZipFile(SRC_ZIP, 'r') as zf:
    txt_members = [n for n in zf.namelist() if n.lower().endswith('.txt')]
    assert len(txt_members) >= 1, "El ZIP no contiene .txt"
    TXT_NAME = txt_members[0]   # usamos el primero
    raw_bytes = zf.read(TXT_NAME)
    raw_io = io.BytesIO(raw_bytes)

# Intento A: lector fiel con quotechar='"'
try:
    dfA = pd.read_csv(
        raw_io,
        sep="\t",
        header=None,
        engine="python",
        dtype=str,
        na_filter=True,
        on_bad_lines="error",
        quotechar='"',
        quoting=csv.QUOTE_MINIMAL
    )
    df_raw = dfA
    print(f"[Plan A] OK filas={len(dfA)} cols={dfA.shape[1]}")
except Exception as eA:
    print("[Plan A] Falló:", eA)
    # reiniciar el buffer para Plan B
    raw_io.seek(0)
    dfB = pd.read_csv(
        raw_io,
        sep="\t",
        header=None,
        engine="python",
        dtype=str,
        na_filter=True,
        on_bad_lines="warn",
        quoting=csv.QUOTE_NONE
    )
    df_raw = dfB
    print(f"[Plan B] OK filas={len(dfB)} cols={dfB.shape[1]}")

# --------- mapeo fijo de columnas (Appendix A: primeras 24) ----------
APPX24 = [
    'RECORD_ID','CAMPNO','MAKETXT','MODELTXT','YEARTXT','MFGCAMPNO','COMPNAME','MFGNAME',
    'BGMAN','ENDMAN','RCLTYPECD','POTAFF','ODATE','INFLUENCED_BY','MFGTXT','RCDATE',
    'DATEA','RPNO','FMVSS','DESC_DEFECT','CONEQUENCE_DEFECT','CORRECTIVE_ACTION',
    'NOTES','RCL_CMPT_ID'
]
ncols = df_raw.shape[1]
df_raw.columns = [f'c{i+1:02d}' for i in range(ncols)]
rename_map = {f'c{i:02d}': APPX24[i-1] for i in range(1, min(24, ncols)+1)}
df = df_raw.rename(columns=rename_map).copy()

# Etiqueta columnas extra (c25..)
for j in range(25, ncols+1):
    c = f'c{j:02d}'
    if c in df.columns:
        df.rename(columns={c: f'EXTRA_{j:02d}'}, inplace=True)

# Corregir typo CONEQUENCE_DEFECT → CONSEQUENCE_DEFECT
if 'CONEQUENCE_DEFECT' in df.columns and 'CONSEQUENCE_DEFECT' not in df.columns:
    df.rename(columns={'CONEQUENCE_DEFECT': 'CONSEQUENCE_DEFECT'}, inplace=True)

# --------- tipificación / normalización EXACTA ----------
# YEARTXT: enteros plausibles (1950..2035), 9999→NaN
if 'YEARTXT' in df.columns:
    df['YEARTXT'] = pd.to_numeric(df['YEARTXT'].replace({'9999': None}), errors='coerce')
    df.loc[(df['YEARTXT'] < 1950) | (df['YEARTXT'] > 2035), 'YEARTXT'] = pd.NA
    df['YEARTXT'] = df['YEARTXT'].astype('Int64')

# Fechas a datetime (formato yyyyMMdd)
for col in ['ODATE','RCDATE','DATEA','BGMAN','ENDMAN']:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], format='%Y%m%d', errors='coerce')

# POTAFF → numérico auxiliar
if 'POTAFF' in df.columns:
    df['POTAFF_num'] = pd.to_numeric(df['POTAFF'], errors='coerce')

# Limpieza superficial de HTML en narrativas y longitud
for c in ['DESC_DEFECT','CONSEQUENCE_DEFECT','CORRECTIVE_ACTION','NOTES']:
    if c in df.columns:
        df[c] = strip_html_series(df[c])
        df[f'len_{c}'] = df[c].str.len()

# Jerarquía COMPNAME → COMP_L1..COMP_L5 (respetando tu criterio de 3, pero dejamos 5 por si acaso)
if 'COMPNAME' in df.columns:
    comp_df = split_component_hierarchy(df['COMPNAME'], max_levels=5)
    df = pd.concat([df, comp_df], axis=1)

# --------- deduplicación por campaña (quédanos el registro más reciente por RCDATE) ----------
if {'CAMPNO','RCDATE'}.issubset(df.columns):
    df = df.sort_values(['CAMPNO','RCDATE'], na_position='last').drop_duplicates('CAMPNO', keep='last')

# --------- agregado por campaña (mantiene consistencia) ----------
agg_spec = {
    'POTAFF_num':'max',
    'MFGNAME':'first',
    'MAKETXT':'first',
    'MODELTXT':'first',
    'COMPNAME':'first',
    'COMP_L1':'first','COMP_L2':'first','COMP_L3':'first','COMP_L4':'first','COMP_L5':'first',
    'RCLTYPECD':'first','FMVSS':'first',
    'RCDATE':'min','ODATE':'min','DATEA':'min',
    'BGMAN':'min','ENDMAN':'max',
    'YEARTXT':'max',
    'DESC_DEFECT':'first','CONSEQUENCE_DEFECT':'first','CORRECTIVE_ACTION':'first','NOTES':'first'
}
present_agg = {k:v for k,v in agg_spec.items() if k in df.columns}
by_camp = df.groupby('CAMPNO', as_index=False).agg(**{k:(k,v) for k,v in present_agg.items()})

# --------- guardados intermedios (trazabilidad) ----------
df.to_csv(PROC_DIR/'recalls_raw_enriched.csv', index=False)
by_camp.to_csv(PROC_DIR/'recalls_by_campaign.csv', index=False)
print("Crudo (dedup):", df.shape, " | Por campaña:", by_camp.shape)

# --------- controles de calidad (idénticos al EDA original) ----------
if {'CAMPNO','POTAFF_num'}.issubset(df.columns):
    var_por_camp = (df.groupby('CAMPNO')['POTAFF_num']
                      .agg(lambda s: pd.Series(s.dropna().unique()).size))
    inconsistentes = var_por_camp[var_por_camp > 1]
    print("Campañas con >1 valor de POTAFF_num:", int((inconsistentes>1).sum()))

for c in ['RCDATE','ODATE','DATEA']:
    if c in by_camp.columns:
        print(f"{c} cobertura:", (by_camp[c].notna().mean()*100).round(2), "%")

text_flags = {}
for c in ['DESC_DEFECT','CONSEQUENCE_DEFECT','CORRECTIVE_ACTION','NOTES']:
    if c in df.columns:
        text_flags[c] = (df[c].fillna('').str.strip().str.len()>0).mean()
if text_flags:
    print("Cobertura de textos (crudo):", {k: round(v*100,1) for k,v in text_flags.items()})

# Outliers y rangos (como antes)
if 'POTAFF_num' in by_camp.columns:
    p = by_camp['POTAFF_num'].dropna()
    print("POTAFF min/max/mediana:", p.min(), p.max(), p.median())
    print("POTAFF <= 0:", int((p <= 0).sum()))
if 'YEARTXT' in by_camp.columns and by_camp['YEARTXT'].notna().any():
    yr = by_camp['YEARTXT'].dropna().astype(int)
    print("YEARTXT min/max:", yr.min(), yr.max())

# --------- export minimal PARA AURA (nombres fijos y jerarquía explicitada) ----------
rec = by_camp.copy()

# Fechas ISO (string yyyy-mm-dd)
for col in ['RCDATE']:
    if col in rec.columns:
        rec[col] = pd.to_datetime(rec[col], errors='coerce').dt.strftime('%Y-%m-%d').fillna('')

# Asegura tipos sencillos
if 'YEARTXT' in rec.columns:
    rec['YEARTXT'] = pd.to_numeric(rec['YEARTXT'], errors='coerce').astype('Int64')

# Renombrado a columnas “amables”:
rename = {
    'CAMPNO':'campaign_no',
    'MAKETXT':'make',
    'MODELTXT':'model',
    'YEARTXT':'year',
    'COMPNAME':'component',
    'RCDATE':'recall_date',
    'DESC_DEFECT':'subject',
    'CONSEQUENCE_DEFECT':'consequence',
    'COMP_L1':'comp_l1',
    'COMP_L2':'comp_l2',
    'COMP_L3':'comp_l3',
    'COMP_L4':'comp_l4',
    'COMP_L5':'comp_l5'
}
for k,v in rename.items():
    if k in rec.columns:
        rec.rename(columns={k:v}, inplace=True)

# Normaliza strings clave (mayúsculas, espacios)
def norm_text_upper(x):
    if not isinstance(x, str): return ''
    s = x.upper().strip()
    s = re.sub(r'\s+', ' ', s)
    # compactar ':' y '/'
    s = s.replace(' :', ':').replace(': ', ':')
    s = re.sub(r'\s*/\s*', '/', s)
    return s

for c in ['campaign_no','make','model','component','subject','consequence','recall_date',
          'comp_l1','comp_l2','comp_l3','comp_l4','comp_l5']:
    if c in rec.columns:
        rec[c] = rec[c].fillna('').map(norm_text_upper)

# Filtro mínimo (misma lógica que antes, PERO sin exigir year para no vaciar)
mask_ok = (
    rec['campaign_no'].str.len().gt(0) &
    rec['make'].str.len().gt(0) &
    rec['model'].str.len().gt(0) &
    rec['comp_l1'].str.len().gt(0)
)
good = rec[mask_ok].copy()

# Si no hay 'component' original, usar comp_l1
if 'component' not in good.columns:
    good['component'] = good['comp_l1']
else:
    good['component'] = np.where(good['component'].str.len().gt(0), good['component'], good['comp_l1'])

# Export final para Neo4j Aura
out_cols = ['campaign_no','make','model','year','component','recall_date','subject','consequence',
            'comp_l1','comp_l2','comp_l3','comp_l4','comp_l5']
for c in out_cols:
    if c not in good.columns:
        good[c] = ''  # rellenar ausentes esperados

out_path = EXP_DIR / 'recalls_neo4j_ready.csv'
good[out_cols].to_csv(out_path, index=False)

print("==================================")
print("Export listo para Aura →", out_path)
print("Filas:", len(good))
print(good[out_cols].head(8))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Plan A] Falló: '	' expected after '"'
[Plan B] OK filas=222573 cols=29
Crudo (dedup): (14340, 39)  | Por campaña: (14340, 23)
Campañas con >1 valor de POTAFF_num: 0
RCDATE cobertura: 100.0 %
ODATE cobertura: 97.75 %
DATEA cobertura: 100.0 %
Cobertura de textos (crudo): {'DESC_DEFECT': np.float64(100.0), 'CONSEQUENCE_DEFECT': np.float64(100.0), 'CORRECTIVE_ACTION': np.float64(100.0), 'NOTES': np.float64(97.3)}
POTAFF min/max/mediana: 0 17600000 466.0
POTAFF <= 0: 6
YEARTXT min/max: 1976 2026
Export listo para Aura → /content/drive/MyDrive/NHTSA/neo4j/exports/recalls_neo4j_ready.csv
Filas: 14340
  campaign_no            make                      model  year  \
0   10C001000       MAXI-COSI                 22-371 HFL  <NA>   
1   10C003000           CYBEX             SOLUTION X-FIX  <NA>   
2   10C005000         EVENFLO              310 (MAESTRO)  <NA>   
3   1

In [ ]:
from neo4j import GraphDatabase
import os

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASS))
CONSTRAINTS = [
  "CREATE CONSTRAINT IF NOT EXISTS FOR (r:Recall)        REQUIRE r.id IS UNIQUE",
  "CREATE CONSTRAINT IF NOT EXISTS FOR (i:Investigation) REQUIRE i.id IS UNIQUE",
  "CREATE CONSTRAINT IF NOT EXISTS FOR (c:Complaint)     REQUIRE c.id IS UNIQUE",
  "CREATE CONSTRAINT IF NOT EXISTS FOR (c:Component)     REQUIRE c.name IS UNIQUE",
  "CREATE CONSTRAINT IF NOT EXISTS FOR (m:Make)          REQUIRE m.name IS UNIQUE",
  "CREATE CONSTRAINT IF NOT EXISTS FOR (m:Model)         REQUIRE (m.name, m.make) IS UNIQUE",
  "CREATE CONSTRAINT IF NOT EXISTS FOR (x:Issue)         REQUIRE x.id IS UNIQUE",
  # índice auxiliar (no único) para búsquedas por minúsculas
  "CREATE INDEX comp_name_lower IF NOT EXISTS FOR (c:Component) ON (c.name_lower)",
]

with driver.session(database="neo4j") as s:
    for q in CONSTRAINTS:
        s.run(q)
        print("OK:", q[:60])
print("Listo ✅")


OK: CREATE CONSTRAINT IF NOT EXISTS FOR (r:Recall)        REQUIR
OK: CREATE CONSTRAINT IF NOT EXISTS FOR (i:Investigation) REQUIR
OK: CREATE CONSTRAINT IF NOT EXISTS FOR (c:Complaint)     REQUIR
OK: CREATE CONSTRAINT IF NOT EXISTS FOR (c:Component)     REQUIR
OK: CREATE CONSTRAINT IF NOT EXISTS FOR (m:Make)          REQUIR
OK: CREATE CONSTRAINT IF NOT EXISTS FOR (m:Model)         REQUIR
OK: CREATE CONSTRAINT IF NOT EXISTS FOR (x:Issue)         REQUIR
OK: CREATE INDEX comp_name_lower IF NOT EXISTS FOR (c:Component)
Listo ✅


In [ ]:
CYPHER_UPSERT_RECALL_HIER_5 = """
UNWIND $rows AS row
// Recall + MM
MERGE (r:Recall {id: row.campaign_no})
  SET r.camp_no     = row.campaign_no,
      r.recall_date = CASE WHEN row.recall_date = '' THEN NULL ELSE row.recall_date END,
      r.make        = row.make,
      r.model       = row.model,
      r.year        = CASE WHEN row.year IS NULL OR row.year = '' THEN NULL ELSE toInteger(row.year) END,
      r.component   = row.component,
      r.subject     = coalesce(row.subject, ''),
      r.consequence = coalesce(row.consequence, '')

MERGE (mk:Make {name: row.make})
MERGE (md:Model {name: row.model, make: row.make})
MERGE (r)-[:OF_MAKE]->(mk)
MERGE (r)-[:OF_MODEL]->(md)

// L1 (siempre existe por el filtro que hicimos)
MERGE (c1:Component {name: row.comp_l1})
  ON CREATE SET c1.name_lower = toLower(row.comp_l1)
  ON MATCH  SET c1.name_lower = coalesce(c1.name_lower, toLower(row.comp_l1))

// L2 (condicional)
FOREACH (_ IN CASE WHEN row.comp_l2 <> '' THEN [1] ELSE [] END |
  MERGE (p1:Component {name: row.comp_l1})
  MERGE (c2:Component {name: row.comp_l2})
    ON CREATE SET c2.name_lower = toLower(row.comp_l2)
    ON MATCH  SET c2.name_lower = coalesce(c2.name_lower, toLower(row.comp_l2))
  MERGE (c2)-[:SUB_OF]->(p1)
)

// L3 (requiere L2 y L3 no vacíos)
FOREACH (_ IN CASE WHEN row.comp_l3 <> '' AND row.comp_l2 <> '' THEN [1] ELSE [] END |
  MERGE (p2:Component {name: row.comp_l2})
  MERGE (c3:Component {name: row.comp_l3})
    ON CREATE SET c3.name_lower = toLower(row.comp_l3)
    ON MATCH  SET c3.name_lower = coalesce(c3.name_lower, toLower(row.comp_l3))
  MERGE (c3)-[:SUB_OF]->(p2)
)

// L4 (requiere L3 y L4)
FOREACH (_ IN CASE WHEN row.comp_l4 <> '' AND row.comp_l3 <> '' THEN [1] ELSE [] END |
  MERGE (p3:Component {name: row.comp_l3})
  MERGE (c4:Component {name: row.comp_l4})
    ON CREATE SET c4.name_lower = toLower(row.comp_l4)
    ON MATCH  SET c4.name_lower = coalesce(c4.name_lower, toLower(row.comp_l4))
  MERGE (c4)-[:SUB_OF]->(p3)
)

// L5 (requiere L4 y L5)
FOREACH (_ IN CASE WHEN row.comp_l5 <> '' AND row.comp_l4 <> '' THEN [1] ELSE [] END |
  MERGE (p4:Component {name: row.comp_l4})
  MERGE (c5:Component {name: row.comp_l5})
    ON CREATE SET c5.name_lower = toLower(row.comp_l5)
    ON MATCH  SET c5.name_lower = coalesce(c5.name_lower, toLower(row.comp_l5))
  MERGE (c5)-[:SUB_OF]->(p4)
)

// Elegimos el componente más profundo disponible y creamos :MENTIONS
WITH r,
     CASE
       WHEN row.comp_l5 <> '' THEN row.comp_l5
       WHEN row.comp_l4 <> '' THEN row.comp_l4
       WHEN row.comp_l3 <> '' THEN row.comp_l3
       WHEN row.comp_l2 <> '' THEN row.comp_l2
       ELSE row.comp_l1
     END AS target_comp
MATCH (cx:Component {name: target_comp})
MERGE (r)-[:MENTIONS]->(cx)

RETURN count(r) AS upserted;

"""


In [ ]:
import pandas as pd
from pathlib import Path

CSV = Path('/content/drive/MyDrive/NHTSA/neo4j/exports/recalls_neo4j_ready.csv')
df = pd.read_csv(CSV, dtype=str, keep_default_na=False)

def ingest(csv_df, cypher, batch=400):
    total, i = len(csv_df), 0
    with driver.session(database="neo4j") as s:
        while i < total:
            rows = csv_df.iloc[i:i+batch].to_dict('records')
            s.run(cypher, rows=rows)
            i += batch
            print(f"→ {min(i,total)}/{total}")
    print("Ingesta completa ✅")

# Test con 200 filas
ingest(df.head(200), CYPHER_UPSERT_RECALL_HIER_5, batch=200)

# Verificación rápida
with driver.session(database="neo4j") as s:
    print("Recalls:", s.run("MATCH (r:Recall) RETURN count(r) AS n").single()['n'])
    print("Components:", s.run("MATCH (c:Component) RETURN count(c) AS n").single()['n'])
    print(s.run("""
      MATCH (r:Recall)-[:MENTIONS]->(c:Component)
      RETURN r.id AS camp, c.name AS comp
      LIMIT 5
    """).data())

# Ingesta total
ingest(df, CYPHER_UPSERT_RECALL_HIER_5, batch=400)

with driver.session(database="neo4j") as s:
    print("Recalls:", s.run("MATCH (r:Recall) RETURN count(r) AS n").single()['n'])
    print("Components:", s.run("MATCH (c:Component) RETURN count(c) AS n").single()['n'])


→ 200/200
Ingesta completa ✅
Recalls: 200
Components: 111
[{'camp': '10C003000', 'comp': 'CHILD SEAT'}, {'camp': '10C005000', 'comp': 'CHILD SEAT'}, {'camp': '10C006000', 'comp': 'CHILD SEAT'}, {'camp': '10E002000', 'comp': 'SUSPENSION'}, {'camp': '10E047000', 'comp': 'SUSPENSION'}]
→ 400/14340
→ 800/14340
→ 1200/14340
→ 1600/14340
→ 2000/14340
→ 2400/14340
→ 2800/14340
→ 3200/14340
→ 3600/14340
→ 4000/14340
→ 4400/14340
→ 4800/14340
→ 5200/14340
→ 5600/14340
→ 6000/14340
→ 6400/14340
→ 6800/14340
→ 7200/14340
→ 7600/14340
→ 8000/14340
→ 8400/14340
→ 8800/14340
→ 9200/14340
→ 9600/14340
→ 10000/14340
→ 10400/14340
→ 10800/14340
→ 11200/14340
→ 11600/14340
→ 12000/14340
→ 12400/14340
→ 12800/14340
→ 13200/14340
→ 13600/14340
→ 14000/14340
→ 14340/14340
Ingesta completa ✅
Recalls: 14340
Components: 509
